# Using Ragas to Evaluate a RAG Application

## Dependencies and API Keys:

In [1]:
import os
from getpass import getpass
from uuid import uuid4
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass("LangSmith API Key:")
os.environ["LANGSMITH_PROJECT"] = f"AIM - RAGAS EVALS - {uuid4().hex[0:8]}"

In [2]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

In [3]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [4]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/39 [00:00<?, ?it/s]

Property 'summary' already exists in node '5cf146'. Skipping!
Property 'summary' already exists in node '303321'. Skipping!
Property 'summary' already exists in node '2d76ab'. Skipping!
Property 'summary' already exists in node 'e5e374'. Skipping!
Property 'summary' already exists in node '5b29f5'. Skipping!
Property 'summary' already exists in node 'ba5f25'. Skipping!
Property 'summary' already exists in node 'f67b6d'. Skipping!
Property 'summary' already exists in node '701435'. Skipping!
Property 'summary' already exists in node '28b8a0'. Skipping!
Property 'summary' already exists in node '6750b3'. Skipping!
Property 'summary' already exists in node 'aca0b2'. Skipping!
Property 'summary' already exists in node '5168d5'. Skipping!
Property 'summary' already exists in node 'e2dd9f'. Skipping!
Property 'summary' already exists in node '644e11'. Skipping!
Property 'summary' already exists in node '59903c'. Skipping!
Property 'summary' already exists in node '089481'. Skipping!
Property

...

Property 'summary_embedding' already exists in node '701435'. Skipping!
Property 'summary_embedding' already exists in node '5cf146'. Skipping!
Property 'summary_embedding' already exists in node '303321'. Skipping!
Property 'summary_embedding' already exists in node 'ba5f25'. Skipping!
Property 'summary_embedding' already exists in node '6750b3'. Skipping!
Property 'summary_embedding' already exists in node 'e5e374'. Skipping!
Property 'summary_embedding' already exists in node '2d76ab'. Skipping!
Property 'summary_embedding' already exists in node '644e11'. Skipping!
Property 'summary_embedding' already exists in node 'f67b6d'. Skipping!
Property 'summary_embedding' already exists in node '59903c'. Skipping!
Property 'summary_embedding' already exists in node '5168d5'. Skipping!
Property 'summary_embedding' already exists in node '5b29f5'. Skipping!
Property 'summary_embedding' already exists in node 'd7becf'. Skipping!
Property 'summary_embedding' already exists in node 'aca0b2'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/8 [00:00<?, ?it/s]

In [5]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,When was ChatGPT launched?,[Introduction ChatGPT launched in November 202...,ChatGPT launched in November 2022.,single_hop_specifc_query_synthesizer
1,Wht do Bick et al. say abot ChatGPT's adopshun...,[Introduction ChatGPT launched in November 202...,Bick et al. (2024) report that 28% of US adult...,single_hop_specifc_query_synthesizer
2,How does ChatGPT usgae differ acros diffrent o...,[Variation by Occupation Figure 23 presents va...,Users in highly paid professional and technica...,single_hop_specifc_query_synthesizer
3,wut is SOC2 code 13 and howw does it relate to...,[Variation by Occupation Figure 23 presents va...,SOC2 code 13 refers to management and business...,single_hop_specifc_query_synthesizer
4,How has the acceleration of artificial intelli...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"The acceleration of artificial intelligence, d...",multi_hop_abstract_query_synthesizer
5,How does the acceleration of artificial intell...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,The acceleration of artificial intelligence is...,multi_hop_abstract_query_synthesizer
6,How does the rapid global adoption of ChatGPT ...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,The rapid global adoption of ChatGPT is eviden...,multi_hop_abstract_query_synthesizer
7,How does the work and non-work usage of ChatGP...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,Work and non-work usage of ChatGPT varies sign...,multi_hop_abstract_query_synthesizer


## LangChain RAG

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

In [6]:
path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=0)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

3122

#### ❓ Question: 

What is the purpose of the `chunk_overlap` parameter in the `RecursiveCharacterTextSplitter`?

Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [9]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="use_case_data",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="use_case_data",
    embedding=embeddings,
)

In [10]:
_ = vector_store.add_documents(documents=split_documents)

In [11]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [12]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### Augmented

Let's create a simple RAG prompt!

In [13]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano")

In [15]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

In [16]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

In [17]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [39]:
response = graph.invoke({"question" : "What are people doing with AI these days?"})

response["response"]

'People are using generative AI at work and outside of work.'

## Evaluating the App with Ragas

In [20]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [21]:
dataset.samples[0].eval_sample.response

'The provided context does not specify an exact launch date for ChatGPT.'

Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [22]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [23]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

Next up - we simply evaluate on our desired metrics!

In [24]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

baseline_result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
baseline_result

Evaluating:   0%|          | 0/48 [00:00<?, ?it/s]

{'context_recall': 0.2589, 'faithfulness': 0.4896, 'factual_correctness': 0.3263, 'answer_relevancy': 0.4701, 'context_entity_recall': 0.2256, 'noise_sensitivity_relevant': 0.1071}

## Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!

> NOTE: This will be using Cohere's Rerank model - please be sure to [sign-up for an API key!](https://docs.cohere.com/reference/about)


We'll first set our retriever to return more documents, which will allow us to take advantage of the reranking.

In [25]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="use_case_data_new_chunks",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="use_case_data_new_chunks",
    embedding=embeddings,
)

_ = vector_store.add_documents(documents=split_documents)

adjusted_example_retriever = vector_store.as_retriever(search_kwargs={"k": 20})

In [26]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=adjusted_example_retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

In [27]:
class AdjustedState(TypedDict):
  question: str
  context: List[Document]
  response: str

adjusted_graph_builder = StateGraph(AdjustedState).add_sequence([retrieve_adjusted, generate])
adjusted_graph_builder.add_edge(START, "retrieve_adjusted")
adjusted_graph = adjusted_graph_builder.compile()

In [37]:
response = adjusted_graph.invoke({"question" : "What are people doing with AI these days?"})

response["response"]

'People are using AI in various ways these days. They seek to use generative AI both at work and outside of work. Users send a large volume of prompts—over 2.5 billion prompts daily to ChatGPT alone—indicating widespread engagement. Conversations with AI are classified into different types based on user intent: asking, doing, or expressing. This suggests that people are using AI for asking questions, performing tasks, and expressing themselves through prompts. Overall, AI technology is being actively used for diverse purposes, including inquiry, task execution, and communication.'

In [29]:
import time
import copy

rerank_dataset = copy.deepcopy(dataset)

for test_row in rerank_dataset:
  response = adjusted_graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(2) # To try to avoid rate limiting.

In [30]:
rerank_dataset.samples[0].eval_sample.response

'ChatGPT was launched in November 2022.'

In [31]:
rerank_evaluation_dataset = EvaluationDataset.from_pandas(rerank_dataset.to_pandas())

In [32]:
rerank_result = evaluate(
    dataset=rerank_evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
rerank_result

Evaluating:   0%|          | 0/48 [00:00<?, ?it/s]

{'context_recall': 0.6518, 'faithfulness': 0.8900, 'factual_correctness': 0.5338, 'answer_relevancy': 0.9523, 'context_entity_recall': 0.4184, 'noise_sensitivity_relevant': 0.1666}

#### ❓ Question: 

Which system performed better, on what metrics, and why?

In [33]:
baseline_result

{'context_recall': 0.2589, 'faithfulness': 0.4896, 'factual_correctness': 0.3263, 'answer_relevancy': 0.4701, 'context_entity_recall': 0.2256, 'noise_sensitivity_relevant': 0.1071}

In [34]:
rerank_result

{'context_recall': 0.6518, 'faithfulness': 0.8900, 'factual_correctness': 0.5338, 'answer_relevancy': 0.9523, 'context_entity_recall': 0.4184, 'noise_sensitivity_relevant': 0.1666}